EXPERIMENT:
- KAIROS uses an encoder decoder architecture trained on benign data
    - 'learning what benign data looks like'
    - low reconstruction error: = benign
    - high reconstruction error: = anomaly, possible threat

- train an encoder decoder on graph node feature vectors:
- a feature vector contains N values sampled from a specific distribution:
    - 'benign' distribution
    - 'drifting' distribution
    - 'drifted' distribution
    - 'threat' distribution

    - the 'benign' distribution has some overlap with the 'drifting' distribution, which has some overlap with the 'drifted' distribution
    - the 'drifted' distribution, the 'benign' distribution, and the 'threat' distribution have no overlap

GOAL:
- As we experience concept drift, we want to learn the new benign behavour while still accurately detecting threats
- Reduce false positives, maintain accuracy
- Also, do we want to minimize performance loss on old, benign data? (depends on the nature of the concept drift, probably out of scope for this project)

Scenario A, NO CONCEPT DRIFT
- initially train on 'benign' only
- evaluate on 'threat' and 'benign' samples
- EXPECTED RESULT: should be pretty good

Scenario B, EFFECTS OF CONCEPT DRIFT
- initially train on 'benign' only
- evaluate on 'drifting' and 'threat' samples
- EXPECTED RESULT: should be worse than A (more false positives)

Scenario C, RE-TRAINING AFTER M NUMBER OF FALSE POSITIVES (specified by human patience) ->  KAIROS
- initially train on 'benign' only
- evaluate on 'drifting' and 'threat' samples (as if deployed in a real detection environment)
- after M number of false positives, re-train or resume training (using data seen thus far)
- evaluate on more (unseen) 'drifting' and 'threat' samples, maybe as a bonus, evaluate on 'drifted'
- IMPORTANT: plot accuracy and false positives over time, mark the point where we re-trained
- EXPECTED RESULT: should be better than B (less false positives)

Scenario D, SUPERVISED LEARNING WITH HUMAN FEEDBACK
- initially train on 'benign' only
- use HUMAN FEEDBACK on 'drifting' and 'threat' samples (as if deployed in a real detection environment), maybe as a bonus, add 'drifted' samples near the end
- IMPORTANT: plot accuracy and false positives over time, mark the point where we started using human feedback
- EXPECTED RESULT: should be better than C (less false positives)




In [1]:
# imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from Data import DistDataset
from Distributions import BENIGN, DRIFTING, DRIFTED, THREAT, is_threat
from tqdm import tqdm

In [2]:
# load our datasets
NodeFeatureLen = 10
NumNodesPerDatasetTRAIN = 20000
NumNodesPerDatasetVAL = 10000
NumNodesPerDatasetTEST = 10000


benign_train_dataset = DistDataset(BENIGN(), NodeFeatureLen, NumNodesPerDatasetTRAIN)
benign_val_dataset = DistDataset(BENIGN(), NodeFeatureLen, NumNodesPerDatasetVAL)
benign_test_dataset = DistDataset(BENIGN(), NodeFeatureLen, NumNodesPerDatasetTEST)


threat_dataset = DistDataset(THREAT(), NodeFeatureLen, NumNodesPerDatasetTEST)

# A_test = torch.utils.data.ConcatDataset([benign_test_dataset[:int(len(benign_test_dataset)/2)], threat_dataset[:int(len(threat_dataset)/2)]])
A_test = torch.utils.data.ConcatDataset([benign_test_dataset, threat_dataset])


# print(len(A_test))


drifting_train_dataset = DistDataset(DRIFTING(), NodeFeatureLen, NumNodesPerDatasetTRAIN)
drifting_test_dataset = DistDataset(DRIFTING(), NodeFeatureLen, NumNodesPerDatasetTEST)

B_test = torch.utils.data.ConcatDataset([drifting_test_dataset, threat_dataset])

# drifted_dataset = DistDataset(DRIFTED(), NodeFeatureLen, NumNodesPerDataset)

# batch_size = 1
# benign_dataloader = DataLoader(benign_dataset, batch_size=batch_size, shuffle=True)
# drifting_dataloader = DataLoader(drifting_dataset, batch_size=batch_size, shuffle=True)
# drifted_dataloader = DataLoader(drifted_dataset, batch_size=batch_size, shuffle=True)
# threat_dataloader = DataLoader(threat_dataset, batch_size=batch_size, shuffle=True)

In [4]:
# define the model architecture to be used for the experiments

class Baseline(nn.Module):
    def __init__(self, NodeFeatureLen):
        super(Baseline, self).__init__()
        self.encode1 = nn.Linear(NodeFeatureLen, 32)
        self.encode2 = nn.Linear(32, 64)
        self.encode3 = nn.Linear(64, 128)

        self.decode1 = nn.Linear(128, 64)
        self.decode2 = nn.Linear(64, 32)
        self.decode3 = nn.Linear(32, NodeFeatureLen)

        # self.encode1 = nn.Linear(NodeFeatureLen, 64)
        # self.encode2 = nn.Linear(64, 32)
        # self.encode3 = nn.Linear(32, 16)

        # self.decode1 = nn.Linear(16, 32)
        # self.decode2 = nn.Linear(32, 64)
        # self.decode3 = nn.Linear(64, NodeFeatureLen)

    def forward(self, x):
        x = F.relu(self.encode1(x))
        x = F.relu(self.encode2(x))
        x = F.relu(self.encode3(x))

        x = F.relu(self.decode1(x))
        x = F.relu(self.decode2(x))
        x = self.decode3(x)
        return x

In [ ]:
# Scenario A, NO CONCEPT DRIFT
# - initially train on 'benign' only
# - evaluate on 'threat' and 'benign' samples
device = 'cuda'
model_A = Baseline(NodeFeatureLen).to(device)
epochs_A = 10
optimizer_A = optim.Adam(model_A.parameters(), lr=1e-3)
batch_size_A = 100


def train_A(model, epochs, trainloader, valloader, optimizer, device):
    print(f"Model has {sum(p.numel() for p in model.parameters())} parameters")
    itrs = tqdm(range(epochs))
    itrs.set_description("Train Loss: ? - Val Loss: ?")
    best_loss = float("inf")

    for epoch in itrs:
        avg_train_loss = 0.0
        batch_num = 0
        for batch in trainloader:
            batch_num+=1
            # print(f"Training on batch {batch_num} / {len(trainloader)}")
            batch_x = batch.to(device)

            optimizer.zero_grad()

            pred = model(batch_x)

            loss = F.mse_loss(pred, batch_x) # reconstruction loss

            loss.backward()
            optimizer.step()

            avg_train_loss += loss.item()

        avg_train_loss /= len(trainloader)

        # validating and saving the model
        with torch.no_grad():
            avg_val_loss = 0.0
            batch_num = 0
            for val_batch in valloader:
                batch_num+=1
                # print(f"Validating on batch {batch_num} / {len(valloader)}")
                batch_val_x = val_batch.to(device)

                pred = model(batch_val_x)

                loss = F.mse_loss(pred, batch_val_x) # reconstruction loss

                avg_val_loss += loss.item()

            avg_val_loss /= len(valloader)

        itrs.set_description(
            f"Train Loss on BENIGN: {avg_train_loss:.3f} - Val Loss on BENIGN: {avg_val_loss:.3f}"
        )

        # print(f"Epoch {epoch+1} Train Loss: {avg_train_loss:.3f} - Val Loss: {avg_val_loss:.3f}")

        # save the model on the best validation loss
        if best_loss > avg_val_loss:
            best_loss = avg_val_loss
            torch.save(model.state_dict(), "Model_A.pt")
            torch.save(optimizer.state_dict(), "Optim_A.pt")

train_A(model_A, 
        epochs_A, 
        trainloader=DataLoader(benign_train_dataset, batch_size=batch_size_A, shuffle=True), 
        valloader=DataLoader(benign_val_dataset, batch_size=batch_size_A, shuffle=True), optimizer=optimizer_A, device=device)


def get_losses(model, testloader, device, string):
    with torch.no_grad():
        avg_val_loss = 0.0
        max_loss = 0
        min_loss = float('inf')

        batch_num = 0
        for val_batch in testloader:
            batch_num+=1
            # print(f"Validating on batch {batch_num} / {len(valloader)}")
            batch_val_x = val_batch.to(device)

            pred = model(batch_val_x)

            loss = F.mse_loss(pred, batch_val_x) # reconstruction loss
            if loss.item() > max_loss:
                max_loss = loss.item()
            if loss.item() < min_loss:
                min_loss = loss.item()

            avg_val_loss += loss.item()

        avg_val_loss /= len(testloader)

    print(f"Max loss on {string}: {max_loss} Min: {min_loss} Avg: {avg_val_loss}")
    return avg_val_loss, max_loss, min_loss


def eval(model, testloader, device, threshold, datasetstr):
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    with torch.no_grad():
        avg_val_loss = 0.0
        batch_num = 0
        for val_batch in testloader:
            batch_num+=1
            # print(f"Validating on batch {batch_num} / {len(valloader)}")
            batch_val_x = val_batch.to(device)

            pred = model(batch_val_x)
            # print("pred ", pred)
            # print("actual ", batch_val_x)
            it = is_threat(batch_val_x)

            loss = F.mse_loss(pred, batch_val_x) # reconstruction loss BUT WE SHOULDN'T USE THIS ON EVAL, instead we should track TP FP TN FN

            

            if loss > threshold: # use the typical loss on the validation set
                # this is a threat
                # if it's actually a threat, TP
                if it:
                    TP+=1
                # else, FP
                else:
                    FP+=1
            else:
                # this is not a threat
                # if it's not actually a threat, TN
                if not it:
                    TN+=1
                # if it's actually a threat, FN
                else:
                    FN+=1

            avg_val_loss += loss.item()

        avg_val_loss /= len(testloader)

    print(f"Test Loss on {datasetstr}: {avg_val_loss:.3f}")
    print(f"TP: {TP}, FP: {FP}, TN: {TN}, FN: {FN}")
        



model_A.load_state_dict(torch.load("Model_A.pt"))
model_A = model_A.to(device)

# determine threshold for benign data
max_loss_on_valid = get_losses(model_A, DataLoader(benign_val_dataset, batch_size=1, shuffle=True), device=device, string="BENIGN VALIDATION SET")[1]


do_not_use2 = get_losses(model_A, DataLoader(drifting_test_dataset, batch_size=1, shuffle=True), device=device, string="DRIFTING TEST SET")[1]
do_not_use = get_losses(model_A, DataLoader(threat_dataset, batch_size=1, shuffle=True), device=device, string="THREAT TEST SET")[1]






Model has 21450 parameters


Train Loss on BENIGN: 0.047 - Val Loss on BENIGN: 0.029: 100%|██████████| 10/10 [00:04<00:00,  2.41it/s]  
C:\Users\Curtis\AppData\Local\Temp\ipykernel_21436\4255333080.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for a

Max loss on BENIGN VALIDATION SET: 4.458788871765137 Min: 0.0064697908237576485 Avg: 0.028678842612029985
Max loss on DRIFTING TEST SET: 3.824460744857788 Min: 0.2288774996995926 Avg: 0.6365728354349732
Max loss on THREAT TEST SET: 34.295982360839844 Min: 14.19006061553955 Avg: 23.161108787059785


In [6]:
eval(model_A, 
       testloader=DataLoader(A_test, batch_size=1, shuffle=True),
       device=device, threshold=1, datasetstr = "BENIGN + THREAT")

Test Loss on BENIGN + THREAT: 11.596
TP: 10000, FP: 37, TN: 9963, FN: 0


In [7]:
# Scenario B
# Scenario B, EFFECTS OF CONCEPT DRIFT
# - initially train on 'benign' only
# - evaluate on 'drifting' and 'threat' samples
# - EXPECTED RESULT: should be worse than A (more false positives)
drifting_threat_loader = DataLoader(B_test, batch_size=1, shuffle=True)
eval(model_A, 
       testloader=drifting_threat_loader,
       device=device, threshold=1, datasetstr="DRIFTING + THREAT")


Test Loss on DRIFTING + THREAT: 11.899
TP: 10000, FP: 51, TN: 9949, FN: 0


In [9]:
# Scenario C
# Scenario C, RE-TRAINING AFTER M NUMBER OF FALSE POSITIVES (specified by human patience) ->  KAIROS
# - initially train on 'benign' only
# - evaluate on 'drifting' and 'threat' samples (as if deployed in a real detection environment)
# - after M number of false positives, re-train or resume training (using data seen thus far)
# - evaluate on more (unseen) 'drifting' and 'threat' samples, maybe as a bonus, evaluate on 'drifted'
# - IMPORTANT: plot accuracy and false positives over time, mark the point where we re-trained
# - EXPECTED RESULT: should be better than B (less false positives)

def train_C(model, epochs, trainloader, optimizer, device):
    print(f"Model has {sum(p.numel() for p in model.parameters())} parameters")
    itrs = tqdm(range(epochs))
    itrs.set_description("Train Loss: ? - Val Loss: ?")
    best_loss = float("inf")
    with torch.set_grad_enabled(True):
        for epoch in itrs:
            avg_train_loss = 0.0
            batch_num = 0
            for batch in trainloader:
                batch_num+=1
                # print(f"Training on batch {batch_num} / {len(trainloader)}")
                batch_x = batch.to(device)

                optimizer.zero_grad()

                pred = model(batch_x)

                loss = F.mse_loss(pred, batch_x) # reconstruction loss

                loss.backward()
                optimizer.step()

                avg_train_loss += loss.item()

            avg_train_loss /= len(trainloader)


            itrs.set_description(
                f"Train Loss on new data: {avg_train_loss:.3f}"
            )
    return model



def eval_C(model, testloader, device, threshold, datasetstr, optim):
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    data_points_for_additional_training = []
    have_re_trained = False
    with torch.no_grad():
        avg_val_loss = 0.0
        batch_num = 0
        for val_batch in testloader:
            batch_num+=1
            # print(f"Validating on batch {batch_num} / {len(valloader)}")
            batch_val_x = val_batch.to(device)

            pred = model(batch_val_x)
            # print("pred ", pred)
            # print("actual ", batch_val_x)
            it = is_threat(batch_val_x)

            loss = F.mse_loss(pred, batch_val_x) # reconstruction loss BUT WE SHOULDN'T USE THIS ON EVAL, instead we should track TP FP TN FN


            if loss > threshold: # use the typical loss on the validation set
                # this is a threat
                # if it's actually a threat, TP
                if it:
                    TP+=1
                # else, FP
                else:
                    data_points_for_additional_training.append(batch_val_x)
                    FP+=1
            else:
                # this is not a threat
                # if it's not actually a threat, TN
                if not it:
                    TN+=1
                    data_points_for_additional_training.append(batch_val_x)
                # if it's actually a threat, FN
                else:
                    FN+=1

            avg_val_loss += loss.item()

            
            
            # check if we hit M number of FP's
            if FP == 25 and not have_re_trained:
                print(f"hit 25 FP at sample number {batch_num}/{len(testloader)}, pausing eval to train some more")
                # then, re-train or resume training on all data seen so far (how many epochs?)
                model = train_C(model, 100, data_points_for_additional_training, optim, device)
                have_re_trained = True
            # then, resume the evaluation

        avg_val_loss /= len(testloader)

    print(f"Test Loss on {datasetstr}: {avg_val_loss:.3f}")
    print(f"TP: {TP}, FP: {FP}, TN: {TN}, FN: {FN}")


model_A.load_state_dict(torch.load("Model_A.pt"))
optimizer_A.load_state_dict(torch.load("Optim_A.pt"))
model_A = model_A.to(device)
eval_C(model_A, 
       testloader=drifting_threat_loader,
       device=device, threshold=1, datasetstr="DRIFTING + THREAT", optim=optimizer_A)


C:\Users\Curtis\AppData\Local\Temp\ipykernel_21436\2865447709.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_A.load_state_dict(torch.load("Model_A.pt"))
C:\User

hit 25 FP at sample number 8066/20000, pausing eval to train some more
Model has 21450 parameters


Train Loss on new data: 5.071:  15%|█▌        | 15/100 [01:29<08:26,  5.96s/it]


KeyboardInterrupt: 

In [ ]:
# Scenario D
# Scenario D, SUPERVISED LEARNING WITH HUMAN FEEDBACK
# - initially train on 'benign' only
# - use HUMAN FEEDBACK on 'drifting' and 'threat' samples (as if deployed in a real detection environment), maybe as a bonus, add 'drifted' samples near the end
# - IMPORTANT: plot accuracy and false positives over time, mark the point where we started using human feedback
# - EXPECTED RESULT: should be better than C (less false positives)

# aka, re-train with one sample X times




def train_D(model, epochs, samples, optimizer, device):
    model = model.train()
    with torch.set_grad_enabled(True):
        for epoch in range(epochs):
            for one_sample in samples:
                # print(f"Training on batch {batch_num} / {len(trainloader)}")
                batch_x = one_sample.to(device)

                optimizer.zero_grad()

                pred = model(batch_x)

                loss = F.mse_loss(pred, batch_x) # reconstruction loss

                loss.backward()
                optimizer.step()

            # print(f"epoch number: {epoch} loss: {loss.item()}")

    model = model.eval()
    return model, optimizer



def eval_D(model, testloader, device, threshold, datasetstr, optim):
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    replay_buffer = []
    with torch.set_grad_enabled(True):
        avg_val_loss = 0.0
        batch_num = 0
        for val_batch in testloader:
            batch_num+=1
            print(f"Validating on batch {batch_num} / {len(testloader)} current FP: {FP}")
            batch_val_x = val_batch.to(device)

            pred = model(batch_val_x)
            # print("pred ", pred)
            # print("actual ", batch_val_x)
            it = is_threat(batch_val_x)

            loss = F.mse_loss(pred, batch_val_x) # reconstruction loss BUT WE SHOULDN'T USE THIS ON EVAL, instead we should track TP FP TN FN


            if loss.item() > threshold: # use the typical loss on the validation set
                # this is a threat
                # if it's actually a threat, TP
                if it:
                    TP+=1
                # else, FP
                else:
                    replay_buffer.append(batch_val_x)
                    if len(replay_buffer) > 10:
                        replay_buffer = replay_buffer[1:]

                    while True:

                        optim.zero_grad()
                        pred = model(batch_val_x)
                        loss2 = F.mse_loss(pred, batch_val_x)
                        loss2.backward()
                        optim.step()

                        if loss2.item() < threshold:
                            break



                    # model, optim = train_D(model, 10, replay_buffer, optim, device)
                    FP+=1
                    # print(FP)
            else:
                # this is not a threat
                # if it's not actually a threat, TN
                if not it:
                    # replay_buffer.append(batch_val_x)
                    # if len(replay_buffer) < 50:
                    #     replay_buffer = replay_buffer[1:]
                    # model, _ = train_D(model, 1, replay_buffer, optim, device)
                    TN+=1
                # if it's actually a threat, FN
                else:
                    FN+=1

            avg_val_loss += loss.item()

        avg_val_loss /= len(testloader)

    print(f"Test Loss on {datasetstr}: {avg_val_loss:.3f}")
    print(f"TP: {TP}, FP: {FP}, TN: {TN}, FN: {FN}")


model_A.load_state_dict(torch.load("Model_A.pt"))
optimizer_A = optim.Adam(model_A.parameters(), lr=1e-7)
# optimizer_A.load_state_dict(torch.load("Optim_A.pt"))
model_A = model_A.to(device)
eval_D(model_A, 
       testloader=drifting_threat_loader,
       device=device, threshold=1, datasetstr="DRIFTING + THREAT", optim=optimizer_A)

# idea, implement replay buffer of a certain length to allow human feedback to make more than 1 update